<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #74d5dd; background-color: #ffffff;">Human Protein Atlas - Single Cell Classification</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">See How To Submit On JUST The Public Part of the Test Set for Easier LB Probing</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

---

Heavily inspired by the discussion posts and notebooks created by [**Darek Kłeczek**](https://www.kaggle.com/thedrcat). I was having difficulty getting the technique he discussed to work, so I created this notebook to take it back to first principles and investigate why it wasn't working.

# 1. Submit Empty Prediction

In [ ]:
import pandas as pd
import numpy as np
import ast

path_to_swap_ss_csv="/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv"
path_to_orig_ss_csv="/kaggle/input/hpasubmission009/sample_submission.csv"

orig_ss_df=pd.read_csv(path_to_orig_ss_csv)
swap_ss_df=pd.read_csv(path_to_swap_ss_csv)

SUBMIT_METHOD = "baseline_2" # One of `empty_1` or `baseline_2`

DEMO_SUBMIT_STYLE=len(swap_ss_df)==559
# DEMO_SUBMIT_STYLE=False

if DEMO_SUBMIT_STYLE:
    orig_ss_df = orig_ss_df[:5]
    swap_ss_df = swap_ss_df[:5]

In [ ]:
# ###################################### #
# # I DON'T KNOW WHY THIS DOESN'T WORK # #
# ###################################### # 
# nan_replacement = orig_ss_df.PredictionString[0] ## GUESSING THIS IS INVALID SOMEHOW
#
# ss_df = swap_ss_df.drop(columns=["PredictionString"]) \
#                   .merge(orig_ss_df.drop(columns=["ImageWidth", "ImageHeight"]), how="left", on="ID") \
#                   .fillna(nan_replacement)
# 
# ss_df.to_csv("/kaggle/working/submission.csv", index=False)
# ###################################### # 

In [ ]:
def create_pred_col(row):
    """ Simple function to return the correct prediction string
    
    We will want the original public test dataframe submission when it is 
    available. However, we will use the swapped inn submission dataframe
    when it is not.
    
    Args:
        row (pd.Series): A row in the dataframe
    
    Returns:
        The prediction string
    """
    if pd.isnull(row.PredictionString_y):
        return row.PredictionString_x
    else:
        return row.PredictionString_y

In [ ]:
ss_df = swap_ss_df.merge(orig_ss_df.drop(columns=["ImageWidth", "ImageHeight"]), how="left", on="ID")
ss_df["PredictionString"] = ss_df.apply(create_pred_col, axis=1)
ss_df = ss_df.drop(columns=["PredictionString_x", "PredictionString_y"])

if SUBMIT_METHOD=="empty_1":
    ss_df.to_csv("/kaggle/working/submission.csv", index=False)
display(ss_df)

# 2. Submit Baseline Prediction

In [ ]:
PUB_SS_W_META_CSV = "/kaggle/input/hpa-sample-submission-with-extra-metadata/updated_sample_submission.csv"
pub_ss_df_w_meta = pd.read_csv(PUB_SS_W_META_CSV)
pub_ss_df_w_meta.mask_rles = pub_ss_df_w_meta.mask_rles.apply(lambda x: ast.literal_eval(x))
pub_ss_df_w_meta.mask_bboxes = pub_ss_df_w_meta.mask_bboxes.apply(lambda x: ast.literal_eval(x))
pub_ss_df_w_meta.mask_sub_rles = pub_ss_df_w_meta.mask_sub_rles.apply(lambda x: ast.literal_eval(x))

display(pub_ss_df_w_meta)

In [ ]:
LABELS_TO_GUESS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
# CONFIDENCES = [0.1,]*len(LABELS_TO_GUESS)
CONFIDENCES = [0.0]*len(LABELS_TO_GUESS)
CONFIDENCES[7]=1.0
print(CONFIDENCES)

pub_ss_df_w_meta.PredictionString = pub_ss_df_w_meta.mask_sub_rles.apply(
    lambda x: " ".join([" ".join([f"{lbl} {conf} {rle}" \
                         for lbl,conf in zip(LABELS_TO_GUESS,CONFIDENCES)]).strip() \
               for rle in x]).strip()
)

pub_ss_df_w_meta

In [ ]:
pub_ss_df_w_meta.PredictionString[0]

In [ ]:
ss_df = swap_ss_df.merge(pub_ss_df_w_meta.drop(
    columns=["ImageWidth", "ImageHeight", "mask_rles", "mask_sub_rles", "mask_bboxes"]
), how="left", on="ID")

ss_df["PredictionString"] = ss_df.apply(create_pred_col, axis=1)
ss_df = ss_df.drop(columns=["PredictionString_x", "PredictionString_y"])

if SUBMIT_METHOD=="baseline_2":
    ss_df.to_csv("/kaggle/working/submission.csv", index=False)
display(ss_df)